In [ ]:
import datetime
import sys
import json
import os


# Load / Save Users (Persistent Storage)


USER_FILE = "users.json"

def load_users():
    if os.path.exists(USER_FILE):
        with open(USER_FILE, "r") as f:
            return json.load(f)
    else:
        # default users
        default_users = {"admin": "admin123"}
        save_users(default_users)
        return default_users

def save_users(users):
    with open(USER_FILE, "w") as f:
        json.dump(users, f)

users = load_users()

current_user = None
current_role = None  # 'admin' or 'user'


# Structures


vendors = [
    {"id": 1, "name": "TechAudio Solutions"},
    {"id": 2, "name": "Visual Systems Inc."},
    {"id": 3, "name": "Event Lighting Pro"}
]

items = [
    {"id": 101, "name": "Wireless Headphones", "vendor_id": 1, "price": 2500},
    {"id": 102, "name": "Bluetooth Speaker", "vendor_id": 1, "price": 1500},
    {"id": 103, "name": "4K Projector", "vendor_id": 2, "price": 5000},
    {"id": 104, "name": "8K LED TV", "vendor_id": 2, "price": 500000},
    {"id": 105, "name": "Stage Spotlight", "vendor_id": 3, "price": 1000}
]

orders = []
cart = []



def clear_screen():
    print("\n" + "=" * 50 + "\n")

def show_header(title):
    print(f"\n--- {title} ---")


# Authentication


def register():
    show_header("Create New Account")

    username = input("Enter username: ")

    if username in users:
        print("Username already exists!")
        return

    password = input("Enter password: ")

    users[username] = password
    save_users(users)

    print("User registered successfully! Please login.")

def login():
    global current_user, current_role

    show_header("Login")

    username = input("Username: ")
    password = input("Password: ")

    if username in users and users[username] == password:
        current_user = username
        current_role = 'admin' if username == 'admin' else 'user'
        print(f"Login Successful! Welcome {username}.")
        input("Press Enter to continue...")
    else:
        print("Invalid credentials.")
        input("Press Enter...")

def logout():
    global current_user, current_role, cart
    current_user = None
    current_role = None
    cart.clear()
    print("Logged out successfully.")


# Vendor Management (Admin)


def maintain_vendors():
    while True:
        clear_screen()
        show_header("Maintain Vendors")

        for v in vendors:
            print(f"ID: {v['id']} | Name: {v['name']}")

        print("\n1. Add Vendor")
        print("2. Remove Vendor")
        print("3. Back")

        choice = input("Select: ")

        if choice == '1':
            name = input("Enter vendor name: ")
            new_id = max(v['id'] for v in vendors) + 1
            vendors.append({"id": new_id, "name": name})
            print("Vendor added.")
            input("Press Enter...")

        elif choice == '2':
            vid = int(input("Enter vendor ID: "))

            if any(i['vendor_id'] == vid for i in items):
                print("Cannot remove vendor with active items.")
            else:
                vendors[:] = [v for v in vendors if v['id'] != vid]
                print("Vendor removed.")

            input("Press Enter...")

        elif choice == '3':
            break


# User Features


def browse_items():
    clear_screen()
    show_header("Browse Items")

    print(f"{'ID':<5} {'Item':<20} {'Vendor':<20} {'Price'}")
    print("-" * 60)

    for item in items:
        vendor_name = next(v['name'] for v in vendors if v['id'] == item['vendor_id'])
        print(f"{item['id']:<5} {item['name']:<20} {vendor_name:<20} ${item['price']}")

    item_id = input("\nEnter Item ID to add to cart (0 to back): ")

    if item_id == '0':
        return

    selected = next((i for i in items if str(i['id']) == item_id), None)

    if selected:
        cart.append(selected)
        print("Item added to cart.")
    else:
        print("Item not found.")

    input("Press Enter...")

def view_cart():
    clear_screen()
    show_header("My Cart")

    if not cart:
        print("Cart is empty.")
        input("Press Enter...")
        return False

    total = 0

    for item in cart:
        print(f"{item['name']} - ${item['price']}")
        total += item['price']

    print("-" * 30)
    print("Total:", total)

    input("Press Enter...")
    return True

def checkout():
    if not cart:
        print("Cart is empty.")
        input("Press Enter...")
        return

    clear_screen()
    show_header("Checkout")

    total = sum(i['price'] for i in cart)
    print("Total Amount:", total)

    print("\nPayment Method:")
    print("1. Cash")
    print("2. UPI")

    method = input("Select: ")
    payment = "Cash" if method == '1' else "UPI"

    order = {
        "order_id": len(orders) + 1001,
        "user": current_user,
        "items": list(cart),
        "total": total,
        "payment": payment,
        "status": "Pending",
        "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    }

    orders.append(order)
    cart.clear()

    print("Order placed successfully!")
    print("Order ID:", order['order_id'])

    input("Press Enter...")

def user_order_status():
    clear_screen()
    show_header("My Orders")

    my_orders = [o for o in orders if o['user'] == current_user]

    if not my_orders:
        print("No orders found.")
    else:
        for o in my_orders:
            print(f"Order #{o['order_id']} | Status: {o['status']} | Total: ${o['total']}")

    input("Press Enter...")


# Admin Order Update


def update_orders():
    clear_screen()
    show_header("Update Orders")

    if not orders:
        print("No orders available.")
        input("Press Enter...")
        return

    for o in orders:
        print(f"ID:{o['order_id']} | User:{o['user']} | Status:{o['status']}")

    oid = input("\nEnter Order ID (0 to back): ")

    if oid == '0':
        return

    order = next((o for o in orders if str(o['order_id']) == oid), None)

    if not order:
        print("Order not found.")
        input("Press Enter...")
        return

    print("\n1. Approved\n2. Shipped\n3. Delivered\n4. Cancelled")

    status_map = {
        '1': "Approved",
        '2': "Shipped",
        '3': "Delivered",
        '4': "Cancelled"
    }

    choice = input("Select status: ")

    if choice in status_map:
        order['status'] = status_map[choice]
        print("Status updated.")
    else:
        print("Invalid choice.")

    input("Press Enter...")


# Main Loop


def main():
    while True:
        clear_screen()
        print("===== Technical Event Management System =====")

        if not current_user:
            print("1. Login")
            print("2. Register")
            print("3. Exit")

            choice = input("Select: ")

            if choice == '1':
                login()
            elif choice == '2':
                register()
            elif choice == '3':
                  print("Exiting system... Goodbye!")
                  break


        else:
            print(f"Logged in as: {current_user} ({current_role})")
            print("-" * 40)

            if current_role == 'user':
                print("1. Browse Items")
                print("2. View Cart")
                print("3. Checkout")
                print("4. Order Status")
                print("5. Logout")

                choice = input("Select: ")

                if choice == '1':
                    browse_items()
                elif choice == '2':
                    view_cart()
                elif choice == '3':
                    checkout()
                elif choice == '4':
                    user_order_status()
                elif choice == '5':
                    logout()

            elif current_role == 'admin':
                print("1. Maintain Vendors")
                print("2. Update Orders")
                print("3. Logout")

                choice = input("Select: ")

                if choice == '1':
                    maintain_vendors()
                elif choice == '2':
                    update_orders()
                elif choice == '3':
                    logout()

if __name__ == "__main__":
    main()




===== Technical Event Management System =====
1. Login
2. Register
3. Exit
